In [1]:
import ee
import geemap

from eelib import eefactory, eefuncs
from pipelines import plcfgs, factories

In [2]:
viewport = ee.FeatureCollection.from_file(
    filename = "../000-data/intersection_64.shp",
)

In [3]:
# load txt file cast to image

with open("./systems.txt", 'r') as file:
    ids = [line.strip() for line in file.readlines()]
ids

['S1B_IW_GRDH_1SDV_20180609T015452_20180609T015517_011290_014BA0_39FD',
 'S1B_IW_GRDH_1SDV_20180609T015517_20180609T015542_011290_014BA0_2658',
 'S1B_IW_GRDH_1SDV_20180609T015542_20180609T015616_011290_014BA0_EE30',
 'S1B_IW_GRDH_1SDV_20180715T015454_20180715T015519_011815_015BE0_0362',
 'S1B_IW_GRDH_1SDV_20180715T015519_20180715T015544_011815_015BE0_D8D7',
 'S1B_IW_GRDH_1SDV_20180715T015544_20180715T015618_011815_015BE0_8287',
 'S1B_IW_GRDH_1SDV_20180913T015503_20180913T015528_012690_0176B4_78B3',
 'S1B_IW_GRDH_1SDV_20180913T015528_20180913T015553_012690_0176B4_0EB4',
 'S1B_IW_GRDH_1SDV_20180913T015553_20180913T015613_012690_0176B4_EB44']

In [4]:
ee_images = [ee.Image(f'COPERNICUS/S1_GRD/{_}') for _ in ids]
ee_images

In [5]:
img_col = ee.ImageCollection(ee_images).map(lambda x: x.set('dateFmt', x.date().format('YYYY-MM-dd')))
dates = img_col.aggregate_array('dateFmt').distinct()
dates_list = dates.getInfo()
# dates_list
filterd = {idx: img_col.filter(f'dateFmt == "{date}"').geometry().dissolve() for idx, date
           in enumerate(dates_list)}


In [6]:
s1_viewport = filterd.get(0).intersection(filterd.get(1)).intersection(filterd.get(2))


In [7]:
datacube = factories.DatacubeCollection(
    arg="projects/fpca-336015/assets/williston-cba"
).filterBounds(s1_viewport).geometry().intersection(s1_viewport)

In [8]:
Map = geemap.Map()


Map



Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [9]:
Map.centerObject(viewport, 6)
Map.addLayer(datacube)
Map.addLayerControl()

In [10]:
viewport_fc = ee.FeatureCollection(datacube).to_file(
    filename="../000-data/viewport.shp"
)